In [1]:
from math import *
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd

#### Loading csv file
Some code functions are taken from the https://www.kaggle.com/code/muhammadqasimshabbir/50sec-runtime-gpu-based-real-estate-demand. It was very helpful in doing the feature engineering

In [2]:
def load_all_data():
    """Load all datasets with optimized memory usage"""
    data = {}
    
    # Main datasets
    datasets = {
        'new': 'data/train/new_house_transactions.csv',
        'new_nb': 'data/train/new_house_transactions_nearby_sectors.csv',
        'pre': 'data/train/pre_owned_house_transactions.csv',
        'pre_nb': 'data/train/pre_owned_house_transactions_nearby_sectors.csv',
        'land': 'data/train/land_transactions.csv',
        'land_nb': 'data/train/land_transactions_nearby_sectors.csv',
        'city_idx': 'data/train/city_indexes.csv',
        'city_search': 'data/train/city_search_index.csv',
        'poi': 'data/train/sector_POI.csv',
        'test': 'data/test.csv'
    }
    
    for name, path in datasets.items():
        try:
            data[name] = pd.read_csv(path)
            print(f"Loaded {name}: {data[name].shape}")
        except Exception as e:
            print(f"Error loading {name}: {e}")
    
    return data




#### FEATURE ENGINEERING

In [86]:
def extract_datetime_features(df, date_col='month'):
    """
    Extract essential datetime features
    """
    if date_col in df.columns:
        # Parse to datetime directly (no splitting needed)
        df['date'] = pd.to_datetime(df[date_col], format='%Y-%b')  # e.g. "2021-Jan"
        df['Year'] = df['date'].dt.year
        df['Month_num'] = df['date'].dt.month
        df['time_index'] = (df['Year'] - df['Year'].min()) * 12 + df['Month_num']
        
        # Seasonality encoding
        df['sin_month'] = np.sin(2 * np.pi * df['Month_num'] / 12)
        df['cos_month'] = np.cos(2 * np.pi * df['Month_num'] / 12)
    
    return df

def extract_sector_features(df, col='sector'):
    """
    Extract numeric sector id from 'sector %d' strings
    """
    if col in df.columns:
        df[col] = df[col].str.extract(r'(\d+)').astype(int)
    return df

def extract_keyword(df, col='keyword'):
    """
    Extracts the keywords and maps them into integers
    """
    keyword_dict = {'买房':1, '二手房市场':2, '公积金':3, '利率上调':4, '去库存':5, '取消限购':6, '契税':7,
                    '学区房':8, '安置':9, '房产税':10, '房价':11, '房价上涨':12, '房价下跌':13, '房价调控':14,
                    '房价走势':15, '房地产开发':16, '房地产税':17, '房屋装修':18, '房贷':19, '棚户区':20,
                    '棚户区改造':21, '租购':22, '税费':23, '落户':24, '融资':25, '购房':26, '贷款利率':27,
                    '限售':28, '限购':29, '首付':30}
    
    if col in df.columns:
        df[col] =  df[col].map(keyword_dict)
    return(df)

def extract_source(df, col='source'):
    """
    Extracts the source of the keyword search and maps into integers
    """

    keyword_dict = {'PC端':1, '移动端':2}
    if col in df.columns:
        df[col] = df[col].map(keyword_dict)
    return(df)

In [110]:
all_data = load_all_data()
all_data['city_idx'] = all_data['city_idx'].iloc[:-1]
print("last row was removed from city_indexes files as it was a copy")
dfname_key = list(all_data.keys())


for key in dfname_key:
    #all_data[key] = extract_datetime_features(all_data[key], date_col='month')
    all_data[key] = extract_sector_features(all_data[key], col='sector')
    all_data[key] = extract_keyword(all_data[key], col='keyword')
    all_data[key] = extract_source(all_data[key], col='source')

#Unique features from all data files
all_features = []
for key in dfname_key:
    temp_keys = all_data[key].keys()
    all_features.extend(temp_keys)

print(len(all_features))
all_features = set(all_features)
all_features = list(all_features)
print(len(all_features) )

Loaded new: (5433, 11)
Loaded new_nb: (5360, 11)
Loaded pre: (5360, 6)
Loaded pre_nb: (5427, 6)
Loaded land: (5896, 6)
Loaded land_nb: (5025, 6)
Loaded city_idx: (7, 74)
Loaded city_search: (4020, 4)
Loaded poi: (86, 142)
Loaded test: (1152, 2)
last row was removed from city_indexes files as it was a copy
268
256


#### MERGING THE DATA FRAMES


In [111]:
def extract_year_features(df, date_col='month'):
    """
    Extract year features
    """
    if date_col in df.columns:
        # Parse to datetime directly (no splitting needed)
        df["year"] = df[date_col].str.split("-").str[0].astype(int)
        
    return df

In [114]:
superdata = pd.merge(all_data['new'], all_data['new_nb'], on=["sector", "month"], how="left")
print(superdata.shape)

superdata = pd.merge(superdata, all_data['pre'], on=["sector", "month"], how="left")
print(superdata.shape)

superdata = pd.merge(superdata, all_data['pre_nb'], on=["sector", "month"], how="left")
print(superdata.shape)

superdata = pd.merge(superdata, all_data['land'], on=["sector", "month"], how="left")
print(superdata.shape)

superdata = pd.merge(superdata, all_data['land_nb'], on=["sector", "month"], how="left")
print(superdata.shape)

superdata = pd.merge(superdata, all_data['poi'], on=["sector"], how="left")
print(superdata.shape)

superdata = pd.merge(superdata, all_data['city_search'], on=["month"], how="left")
print(superdata.shape)

#NOTICE THE SHAPE CHANGE OF THE SUPER DATA SET HERE
superdata = extract_year_features(superdata, date_col='month')
print(superdata.shape)
print(superdata['year'].unique())

#Changing the feature name to year of the city(not sector)
all_data['city_idx'].rename(columns={'city_indicator_data_year': 'year'}, inplace=True)
superdata = pd.merge(superdata, all_data['city_idx'], on=["year"], how="left")
print(superdata.shape)
print(superdata['year'].unique())

#Filling NaN with 0 in the entire data frame
superdata = superdata.fillna(0)
superdata = extract_datetime_features(superdata, date_col='month')

superdata.drop('Year', axis=1, inplace=True)
superdata.drop('month', axis=1, inplace=True)
first_row = superdata.iloc[0]

#CHECKING FOR THE FEATURES HAVE STRING ENTRIES
string_check = 0
for col, value in first_row.items():
    if isinstance(value, str):
        print(col, isinstance(value, str))
        string_check = 1

if string_check==0:
    print("No string in the data!! Proceed further for training")
else:
    print("Data not ready for training")
    

(5433, 20)
(5433, 24)
(5433, 28)
(5433, 32)
(5433, 36)
(5433, 177)
(325980, 180)
(325980, 181)
[2019 2020 2021 2022 2023 2024]
(325980, 254)
[2019 2020 2021 2022 2023 2024]
No string in the data!! Proceed further for training


In [96]:
print(superdata['year'].unique())



[2019 2020 2021 2022 2023 2024]
